# Exercise 15

# Fraud Detection

## Introduction

- Fraud Detection Dataset from Microsoft Azure: [data](http://gallery.cortanaintelligence.com/Experiment/8e9fe4e03b8b4c65b9ca947c72b8e463)

Fraud detection is one of the earliest industrial applications of data mining and machine learning. Fraud detection is typically handled as a binary classification problem, but the class population is unbalanced because instances of fraud are usually very rare compared to the overall volume of transactions. Moreover, when fraudulent transactions are discovered, the business typically takes measures to block the accounts from transacting to prevent further losses. 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [2]:
import zipfile
with zipfile.ZipFile('15_fraud_detection.csv.zip', 'r') as z:
    f = z.open('15_fraud_detection.csv')
    data = pd.io.parsers.read_table(f, index_col=0, sep=',')

In [3]:
data.head()

,accountAge,digitalItemCount,sumPurchaseCount1Day,sumPurchaseAmount1Day,sumPurchaseAmount30Day,paymentBillingPostalCode - LogOddsForClass_0,accountPostalCode - LogOddsForClass_0,paymentBillingState - LogOddsForClass_0,accountState - LogOddsForClass_0,paymentInstrumentAgeInAccount,ipState - LogOddsForClass_0,transactionAmount,transactionAmountUSD,ipPostalCode - LogOddsForClass_0,localHour - LogOddsForClass_0,Label
0,2000,0,0,0.00,720.25,5.064533,0.421214,1.312186,0.566395,3279.574306,1.218157,599.00,626.164650,1.259543,4.745402,0
1,62,1,1,1185.44,2530.37,0.538996,0.481838,4.401370,4.500157,61.970139,4.035601,1185.44,1185.440000,3.981118,4.921349,0
2,2000,0,0,0.00,0.00,5.064533,5.096396,3.056357,3.155226,0.000000,3.314186,32.09,32.090000,5.008490,4.742303,0
3,1,1,0,0.00,0.00,5.064533,5.096396,3.331154,3.331239,0.000000,3.529398,133.28,132.729554,1.324925,4.745402,0
4,1,1,0,0.00,132.73,5.412885,0.342945,5.563677,4.086965,0.001389,3.529398,543.66,543.660000,2.693451,4.876771,0


In [4]:
data.shape, data.Label.sum(), data.Label.mean()

((138721, 16), 797, 0.0057453449730033666)

# Exercice 15.1

Estimate a Logistic Regression and a Decision Tree

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=10)

Comment about the results

In [5]:
X = data.drop('Label',axis=1)
y = data.Label

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [6]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [7]:
# make predictions for testing set
y_pred_class = logreg.predict(X_test)

# calculate testing accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.994002479744


In [8]:
from sklearn.tree import DecisionTreeClassifier
treeclf = DecisionTreeClassifier()
treeclf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [9]:
y_pred_tree = treeclf.predict(X_test)

# calculate testing accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_pred_tree, y_test))

0.98817796488


In [10]:
metrics.confusion_matrix(y_pred_tree,y_test)

array([[34241,   178],
       [  232,    30]])

In [11]:
metrics.confusion_matrix(y_pred_class,y_test)

array([[34473,   208],
       [    0,     0]])

In [12]:
metrics.precision_score(y_pred_tree,y_test),metrics.recall_score(y_pred_tree,y_test)

(0.14423076923076922, 0.11450381679389313)

In [13]:
metrics.f1_score(y_pred_tree,y_test)

0.12765957446808507

In [14]:
metrics.fbeta_score(y_pred_tree,y_test,beta=10)

0.11473795819448651

La matriz de confusión nos permite ver que los árboles de decisión están prediciendo mayor número de  falsos  postivos, la regresión logística menor cantidad de falsos positivos. Se puede observar que el modelo logístico posee un nive de predicción  del 99% mientras que los árboles de decisión 98%.

# Exercice 15.2 (2 points)

Under-sample the negative class using random-under-sampling

Which is parameter for target_percentage did you choose?
How the results change?

**Only apply under-sampling to the training set, evaluate using the whole test set**

In [15]:
def UnderSampling(X, y, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()

    n_samples_0_new =  n_samples_1 / target_percentage - n_samples_1
    n_samples_0_new_per = n_samples_0_new / n_samples_0

    filter_ = y == 0

    np.random.seed(seed)
    rand_1 = np.random.binomial(n=1, p=n_samples_0_new_per, size=n_samples)
    
    filter_ = filter_ & rand_1
    filter_ = filter_ | (y == 1)
    filter_ = filter_.astype(bool)
    
    return X[filter_], y[filter_]

In [16]:
for target_percentage in [0.1, 0.2, 0.3, 0.4, 0.5]:
    X_u, y_u = UnderSampling(X_train, y_train, target_percentage, 666)
    print('Target percentage', target_percentage)
    print('y.shape = ',y_u.shape[0], 'y.mean() = ', y_u.mean())

Target percentage 0.1
y.shape =  5973 y.mean() =  0.0986104135275406
Target percentage 0.2
y.shape =  3019 y.mean() =  0.19509771447499172
Target percentage 0.3
y.shape =  2022 y.mean() =  0.291295746785361
Target percentage 0.4
y.shape =  1484 y.mean() =  0.396900269541779
Target percentage 0.5
y.shape =  1205 y.mean() =  0.4887966804979253


In [17]:
X_u, y_u = UnderSampling(X_train, y_train, 0.5, 666)

In [18]:
X_u.shape,y_u.shape

((1205, 15), (1205,))

In [19]:
logreg.fit(X_u, y_u)
treeclf.fit(X_u,y_u)

y_pred_class_log = logreg.predict(X_test)
y_pred_class_tree = treeclf.predict(X_test)

In [20]:
metrics.accuracy_score(y_pred_class_log,y_test), metrics.accuracy_score(y_pred_class_tree,y_test)

(0.61062253106888498, 0.67844064473342758)

In [22]:
metrics.precision_score(y_pred_tree,y_test), metrics.recall_score(y_pred_tree,y_test)

(0.14423076923076922, 0.11450381679389313)

In [23]:
metrics.f1_score(y_pred_tree,y_test)

0.12765957446808507

In [24]:
metrics.fbeta_score(y_pred_tree,y_test,beta=10)

0.11473795819448651

# Exercice 15.3 (2 points)

Same analysis using TomekLinks and Condensed Nearest Neighbours

Do not test different parameters for CNN

In [25]:
from sklearn.neighbors import KNeighborsClassifier
def CondensedNearestNeighbor(X, y, n_seeds_S=1, size_ngh=1, seed=None):
    # Randomly get one sample from the majority class
    np.random.seed(seed)
    maj_sample = np.random.choice(X[y == 0].shape[0], n_seeds_S)
    maj_sample = X[y == 0][maj_sample]
    # Create the set C
    # Select all positive and the randomly selected negatives
    C_x = np.append(X[y == 1], maj_sample, axis=0)
    C_y = np.append(y[y == 1], [0] * n_seeds_S)
    # Create the set S
    S_x = X[y == 0]
    S_y = y[y == 0]
    knn = KNeighborsClassifier(n_neighbors=size_ngh)

    # Fit C into the knn
    knn.fit(C_x, C_y)

    # Classify on S
    pred_S_y = knn.predict(S_x)
    # Find the misclassified S_y
    idx_tmp = np.nonzero(y == 0)[0][np.nonzero(pred_S_y != S_y)]

    filter_ = np.nonzero(y == 1)[0]
    filter_ = np.concatenate((filter_, idx_tmp), axis=0)

    return X[filter_], y[filter_]

In [43]:
for n_seeds_S, size_ngh in [(1, 1), (100, 100), (50, 50), (100, 50), (50, 100)]:
    X_u, y_u = CondensedNearestNeighbor(X_train, y_train, n_seeds_S, size_ngh, 1)

IndexError: indices are out-of-bounds

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

lru =LogisticRegression()
dtu = DecisionTreeClassifier()

lru.fit(X_u,y_u)
dtu.fit(X_u,y_u)

y_pred_class_log = lru.predict(X_test)
y_pred_class_tree = dtu.predict(X_test)

metrics.accuracy_score(y_pred_class_log,y_test), metrics.accuracy_score(y_pred_class_tree,y_test)

(0.60877714022087026, 0.69334794267754674)

In [29]:
metrics.precision_score(y_pred_class_tree,y_test), metrics.recall_score(y_pred_class_tree,y_test)

(0.66346153846153844, 0.01289358123890498)

In [30]:
metrics.f1_score(y_pred_class_tree,y_test)

0.025295573274676932

In [42]:
metrics.fbeta_score(y_pred_class_tree,y_test,beta=10)

0.013019986772635052

# Exercice 15.4 

Now using random-over-sampling

In [36]:
import random
def OverSampling(X, y, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()

    n_samples_1_new =  -target_percentage * n_samples_0 / (target_percentage- 1)

    np.random.seed(seed)
    filter_ = np.random.choice(X[y == 1].shape[0], int(n_samples_1_new))
    # filter_ is within the positives, change to be of all
    filter_ = np.nonzero(y == 1)[0][filter_]
    
    filter_ = np.concatenate((filter_, np.nonzero(y == 0)[0]), axis=0)
    
    return X[filter_], y[filter_]

In [37]:
for target_percentage in [0.1, 0.2, 0.3, 0.4, 0.5]:
    X_u, y_u = OverSampling(X_train, y_train, target_percentage, 1)

IndexError: indices are out-of-bounds

# Exercice 15.5 (3 points)

Evaluate the results using SMOTE

Which parameters did you choose?

In [38]:
def SMOTE(X, y, target_percentage=0.5, k=5, seed=None):
    
    # New samples
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()
    n_samples_1_new =  int(-target_percentage * n_samples_0 / (target_percentage- 1) - n_samples_1)
    
    # A matrix to store the synthetic samples
    new = np.zeros((n_samples_1_new, X.shape[1]))
    
    # Create seeds
    np.random.seed(seed)
    seeds = np.random.randint(1, 1000000, 3)
    
    # Select examples to use as base
    np.random.seed(seeds[0])
    sel_ = np.random.choice(y[y==1].shape[0], n_samples_1_new)
    
    # Define random seeds (2 per example)
    np.random.seed(seeds[1])
    nn__ = np.random.choice(k, n_samples_1_new)
    np.random.seed(seeds[2])
    steps = np.random.uniform(size=n_samples_1_new)  

    # For each selected examples create one synthetic case
    for i, sel in enumerate(sel_):
        # Select neighbor
        nn_ = nn__[i]
        step = steps[i]
        # Create new sample
        new[i, :] = X[y==1][sel] - step * (X[y==1][sel] - X[y==1][nn_])
    
    X = np.vstack((X, new))
    y = np.append(y, np.ones(n_samples_1_new))
    
    return X, y

In [ ]:
import numpy as np
for target_percentage in [0.25, 0.5]:
    for k in [5, 15]:
        X_u, y_u = SMOTE(X_train.values, y_train, target_percentage, k, seed=666)
        print('Target percentage', target_percentage, 'k ', k)
        print('y.shape = ',y_u.shape[0], 'y.mean() = ', y_u.mean())

/Users/FelipeGarcia/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:32: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


In [ ]:
X_u, y_u = SMOTE(X_train, y_train, 0.1, 5, seed=666)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

lru =LogisticRegression()
dtu = DecisionTreeClassifier()

lru.fit(X_u,y_u)
dtu.fit(X_u,y_u)

y_pred_class_log = lru.predict(X_test)
y_pred_class_tree = dtu.predict(X_test)

In [ ]:
metrics.fbeta_score(y_pred_class_log,y_test,beta=10),metrics.fbeta_score(y_pred_class_tree,y_test,beta=10)

In [ ]:
metrics.accuracy_score(y_pred_class_log,y_test), metrics.accuracy_score(y_pred_class_tree,y_test)

# Exercice 15.6 (3 points)

Compare and comment about the results